In [ ]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
# Load the CSV (replace 'your_csv.csv' with your actual file)
df = pd.read_csv('/content/train.csv')
# Add a serial number column
df.insert(0, 'Serial Number', range(1, len(df) + 1))

In [ ]:
print(df.head())

   Serial Number  Unnamed: 0        DATE  LATITUDE_A  LONGITUDE_A  \
0              1           0  01-11-1979       63.88        77.52   
1              2           1  13-11-1979       63.88        77.52   
2              3           2  05-02-1980       63.88        77.52   
3              4           3  23-12-1979       63.88        77.52   
4              5           4  07-06-1977       63.88        77.52   

   ELEVATION_A  PRCP_A  SNWD_A  TMAX_A  TMIN_A  ...  LONGITUDE_C  ELEVATION_C  \
0          128     NaN     0.0    17.6     NaN  ...        78.93           64   
1          128     NaN     0.0     8.8     NaN  ...        78.93           64   
2          128     NaN     0.0    -9.9     NaN  ...        78.93           64   
3          128     0.0     0.0     NaN   -4.30  ...        78.93           64   
4          128     NaN     0.0    39.6   15.48  ...        78.93           64   

   PRCP_C  SNWD_C  TMAX_C  TMIN_C  TAVG_C  LATITUDE  LONGITUDE  TAVG  
0     0.0    0.00    18.7  

In [ ]:
# Split the date into day, month, and year
df['Date'] = pd.to_datetime(df['DATE'], dayfirst=True)
df['Day'] = df['Date'].dt.day
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

# Drop the original date column
df.drop('DATE', axis=1, inplace=True)
# Sort the dataset using day, month, and year
df.sort_values(by=['Year', 'Month', 'Day'], inplace=True)
df.drop(['Date', 'Unnamed: 0'], axis=1, inplace=True)

In [ ]:
print(df.head())

     Serial Number  LATITUDE_A  LONGITUDE_A  ELEVATION_A  PRCP_A  SNWD_A  \
147            148       63.88        77.52          128     NaN    0.00   
641            642       63.88        77.52          128     NaN    0.00   
332            333       63.88        77.52          128     0.0    2.89   
163            164       63.88        77.52          128     NaN    0.00   
243            244       63.88        77.52          128     NaN    0.00   

     TMAX_A  TMIN_A  TAVG_A  LATITUDE_B  ...  SNWD_C  TMAX_C  TMIN_C  TAVG_C  \
147    34.1     NaN   68.16       62.95  ...     0.0     NaN     NaN     NaN   
641     NaN     NaN   29.40       62.95  ...     0.0     NaN     NaN    29.4   
332    34.1     NaN   30.24       62.95  ...     0.0    36.3     NaN    33.6   
163    24.2     NaN   15.60       62.95  ...     0.0     NaN     NaN     NaN   
243    -2.2     NaN   -4.80       62.95  ...     0.0     NaN     NaN    -5.4   

     LATITUDE  LONGITUDE  TAVG  Day  Month  Year  
147    63.9

In [ ]:
# Impute missing values
imputer = KNNImputer(n_neighbors=5)
df_train_imputed_numeric = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

In [ ]:
print(df_train_imputed_numeric.head())

   Serial Number  LATITUDE_A  LONGITUDE_A  ELEVATION_A  PRCP_A  SNWD_A  \
0          148.0       63.88        77.52        128.0     0.0    0.00   
1          642.0       63.88        77.52        128.0     0.0    0.00   
2          333.0       63.88        77.52        128.0     0.0    2.89   
3          164.0       63.88        77.52        128.0     0.0    0.00   
4          244.0       63.88        77.52        128.0     0.0    0.00   

   TMAX_A  TMIN_A  TAVG_A  LATITUDE_B  ...  SNWD_C  TMAX_C  TMIN_C  TAVG_C  \
0   34.10  12.900   68.16       62.95  ...     0.0  34.760  19.856  27.072   
1   37.18  22.360   29.40       62.95  ...     0.0  51.436  14.630  29.400   
2   34.10  30.600   30.24       62.95  ...     0.0  36.300  17.480  33.600   
3   24.20  14.826   15.60       62.95  ...     0.0  30.140  11.590  19.992   
4   -2.20  -4.438   -4.80       62.95  ...     0.0   6.842  -3.478  -5.400   

   LATITUDE  LONGITUDE  TAVG   Day  Month    Year  
0    63.933     78.733  24.7   9.0

In [ ]:
# Sort back to the original order using the serial number
df_train_imputed_numeric.sort_values(by='Serial Number', inplace=True)
df_train_imputed_numeric.drop('Serial Number', axis=1, inplace=True)

In [ ]:
print(df_train_imputed_numeric.head())

     LATITUDE_A  LONGITUDE_A  ELEVATION_A  PRCP_A  SNWD_A  TMAX_A  TMIN_A  \
669       63.88        77.52        128.0     0.0     0.0    17.6  16.546   
675       63.88        77.52        128.0     0.0     0.0     8.8  -2.512   
716       63.88        77.52        128.0     0.0     0.0    -9.9  -3.028   
691       63.88        77.52        128.0     0.0     0.0     3.3  -4.300   
247       63.88        77.52        128.0     0.0     0.0    39.6  15.480   

     TAVG_A  LATITUDE_B  LONGITUDE_B  ...  SNWD_C  TMAX_C  TMIN_C  TAVG_C  \
669  12.960       62.95        79.82  ...    0.00   18.70   1.520  19.344   
675   0.000       62.95        79.82  ...    0.00   11.00  -3.800   2.040   
716 -17.760       62.95        79.82  ...    0.00   -2.64  -1.938 -16.560   
691  -4.440       62.95        79.82  ...    0.00    0.00   8.038  -2.640   
247  38.616       62.95        79.82  ...    3.82   38.50  17.100  33.000   

     LATITUDE  LONGITUDE  TAVG   Day  Month    Year  
669    63.933     78

In [ ]:
# Save the imputed training data
output_file_path = '/content/knn_and_sorted_train.csv'
df_train_imputed_numeric.to_csv(output_file_path, index=False, index_label='INDEX')

In [ ]:
# Load and process test CSV
df = pd.read_csv('/content/test.csv')

In [ ]:
print(df.head())

   INDEX        DATE  LATITUDE_A  LONGITUDE_A  ELEVATION_A  PRCP_A  SNWD_A  \
0      0  04-11-1978      63.883       77.517          128     0.0     0.0   
1      1  11-04-1980      63.883       77.517          128     NaN     0.0   
2      2  29-04-1977      63.883       77.517          128     NaN     0.0   
3      3  12-10-1977      63.883       77.517          128     0.0     0.0   
4      4  02-09-1980      63.883       77.517          128     NaN     0.0   

   TMAX_A  TMIN_A  TAVG_A  ...  LATITUDE_C  LONGITUDE_C  ELEVATION_C  PRCP_C  \
0     5.0     NaN     2.3  ...      63.083       78.933           64     NaN   
1     NaN     NaN     9.3  ...      63.083       78.933           64     0.0   
2     NaN     NaN    18.0  ...      63.083       78.933           64     0.0   
3    16.0     NaN     9.8  ...      63.083       78.933           64     0.0   
4    29.0    10.0    18.8  ...      63.083       78.933           64   500.0   

   SNWD_C  TMAX_C  TMIN_C  TAVG_C  LATITUDE  LONGI

In [ ]:
# Add a serial number column
df.insert(0, 'Serial Number', range(1, len(df) + 1))

# Split the date into day, month, and year
df['Date'] = pd.to_datetime(df['DATE'], dayfirst=True)
df['Day'] = df['Date'].dt.day
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

# Drop the original date column
df.drop('DATE', axis=1, inplace=True)
# Sort the dataset using day, month, and year
df.sort_values(by=['Year', 'Month', 'Day'], inplace=True)
df.drop(['Date'], axis=1, inplace=True)

# Impute missing values
imputer = KNNImputer(n_neighbors=5)
df_test_imputed_numeric = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

# Sort back to the original order using the serial number
df_test_imputed_numeric.sort_values(by='Serial Number', inplace=True)
df_test_imputed_numeric.drop(['Serial Number', 'INDEX'], axis=1, inplace=True)

In [ ]:
print(df_test_imputed_numeric.head())

     LATITUDE_A  LONGITUDE_A  ELEVATION_A  PRCP_A  SNWD_A  TMAX_A  TMIN_A  \
117      63.883       77.517        128.0     0.0     0.0     5.0     2.8   
187      63.883       77.517        128.0     0.0     0.0    23.4    11.8   
51       63.883       77.517        128.0     0.0     0.0    23.2    11.8   
67       63.883       77.517        128.0     0.0     0.0    16.0    11.8   
202      63.883       77.517        128.0     1.0     0.0    29.0    10.0   

     TAVG_A  LATITUDE_B  LONGITUDE_B  ...  PRCP_C  SNWD_C  TMAX_C  TMIN_C  \
117     2.3       62.95      79.8167  ...     0.0    0.00     8.0     0.6   
187     9.3       62.95      79.8167  ...     0.0    0.00    28.0     8.0   
51     18.0       62.95      79.8167  ...     0.0    2.89    30.0    10.4   
67      9.8       62.95      79.8167  ...     0.0    0.00    19.0     6.8   
202    18.8       62.95      79.8167  ...   500.0    0.00    30.0    11.6   

     TAVG_C  LATITUDE  LONGITUDE   Day  Month    Year  
117     3.0    63.

In [ ]:
# Save the imputed test data
output_file_path = '/content/knn_and_sorted_test.csv'
df_test_imputed_numeric.to_csv(output_file_path, index=False, index_label='INDEX')

In [ ]:
# Load the imputed training data
train_file_path = '/content/knn_and_sorted_train.csv'
df_train = pd.read_csv(train_file_path)

In [ ]:
# Feature Engineering
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of Earth in kilometers
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) * 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) * 2
    return 2 * R * np.arcsin(np.sqrt(a))

df_train['DIST_A'] = haversine(df_train['LATITUDE'], df_train['LONGITUDE'], df_train['LATITUDE_A'], df_train['LONGITUDE_A'])
df_train['DIST_B'] = haversine(df_train['LATITUDE'], df_train['LONGITUDE'], df_train['LATITUDE_B'], df_train['LONGITUDE_B'])
df_train['DIST_C'] = haversine(df_train['LATITUDE'], df_train['LONGITUDE'], df_train['LATITUDE_C'], df_train['LONGITUDE_C'])

# Define the feature set and target for training data
y_train = df_train['TAVG']
df_train.drop(['TAVG'], axis=1, inplace=True)
X_train = df_train

<ipython-input-47-22390d784fd5>:8: RuntimeWarning: invalid value encountered in sqrt
  return 2 * R * np.arcsin(np.sqrt(a))
<ipython-input-47-22390d784fd5>:8: RuntimeWarning: invalid value encountered in sqrt
  return 2 * R * np.arcsin(np.sqrt(a))
<ipython-input-47-22390d784fd5>:8: RuntimeWarning: invalid value encountered in sqrt
  return 2 * R * np.arcsin(np.sqrt(a))


In [ ]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [ ]:
# Initialize and train the RandomForestRegressor
rf_model = RandomForestRegressor(
    n_estimators=723,
    criterion="squared_error",
    max_depth=22,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features=None,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=42,
    verbose=1,
    warm_start=False,
    ccp_alpha=0.0,
    max_samples=None)
rf_model.fit(X_train_scaled, y_train)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    1.3s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    2.8s


RandomForestRegressor(max_depth=22, max_features=None, n_estimators=723,
                      random_state=42, verbose=1)

In [ ]:
# Evaluate the model on validation data
y_val_pred = rf_model.predict(X_val_scaled)
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
val_mae = mean_absolute_error(y_val, y_val_pred)

print('Validation Results:')
print(f'RMSE: {val_rmse}')
print(f'MAE: {val_mae}')

Validation Results:
RMSE: 2.1812031686099878
MAE: 1.5649118787601084


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.1s


In [ ]:
# Testing part of your code remains the same, following the same preprocessing steps and predictions.

test_file_path = '/content/knn_and_sorted_test.csv'
df_test = pd.read_csv(test_file_path)

df_test['DIST_A'] = haversine(df_test['LATITUDE'], df_test['LONGITUDE'], df_test['LATITUDE_A'], df_test['LONGITUDE_A'])
df_test['DIST_B'] = haversine(df_test['LATITUDE'], df_test['LONGITUDE'], df_test['LATITUDE_B'], df_test['LONGITUDE_B'])
df_test['DIST_C'] = haversine(df_test['LATITUDE'], df_test['LONGITUDE'], df_test['LATITUDE_C'], df_test['LONGITUDE_C'])

# Scale the test data
X_test_scaled = scaler.transform(df_test)

<ipython-input-11-22390d784fd5>:8: RuntimeWarning: invalid value encountered in sqrt
  return 2 * R * np.arcsin(np.sqrt(a))
<ipython-input-11-22390d784fd5>:8: RuntimeWarning: invalid value encountered in sqrt
  return 2 * R * np.arcsin(np.sqrt(a))
<ipython-input-11-22390d784fd5>:8: RuntimeWarning: invalid value encountered in sqrt
  return 2 * R * np.arcsin(np.sqrt(a))


In [ ]:
# Predict on the test data
y_test_pred = rf_model.predict(X_test_scaled)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.0s


In [ ]:
# Add the predicted TAVG as a new column to the original test DataFrame
df_test['TAVG'] = y_test_pred

# Save the updated test DataFrame with only index and TAVG columns
df_output = df_test[['TAVG']]  # Select only the 'TAVG' column

In [ ]:
output_file_path = '/content/test_with_predicted_TAVG.csv'
df_output.to_csv(output_file_path, index=True, index_label='INDEX')  # Save with index as 'INDEX' column